In [1]:
# If you don't have one of these required libraries, pip install them
import os
import sys
# Need for local fhirclient install
sys.path.append('/home/idies/workspace/Storage/showson1/persistent/client-py')

In [2]:
import sqlalchemy
import urllib.parse
import pandas as pd
import numpy as np
import getpass
from dataclasses import dataclass
from SciServer import Authentication
from fhirclient.models.clinicalprofile import *
from datetime import datetime
import json
from fhir_loader import fhir_loader
from fhirclient.models import *
import pymssql

In [3]:
user = 'clinicalprofile'
passwd = getpass.getpass('Password for '+user+':')

Password for clinicalprofile:········


In [4]:
driver='FreeTDS'
tds_ver='8.0'

host_ip='esmpmdbdev6.database.windows.net'
db_port=1433
db='ClinicalProfile'

conn_str=('DRIVER={};SERVER={};PORT={};DATABASE={};UID={};PWD={};TDS_VERSION={}'.format(
driver, host_ip, db_port, db, user, passwd, tds_ver))

engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect='+urllib.parse.quote(conn_str))

query = "select * from sys.views"
pd.read_sql_query(query, engine)

,name,object_id,principal_id,schema_id,parent_object_id,type,type_desc,create_date,modify_date,is_ms_shipped,is_published,is_schema_published,is_replicated,has_replication_filter,has_opaque_metadata,has_unchecked_assembly_data,with_check_option,is_date_correlation_view,is_tracked_by_cdc,has_snapshot
0,vw_cp_labs,18099105,None,1,0,V,VIEW,2019-04-15 19:37:12.953333,2019-06-21 15:38:39.006666,False,False,False,False,False,False,False,False,False,False,False
1,vw_pc_dm_asthma_demo,34099162,None,1,0,V,VIEW,2019-04-15 20:01:55.696666,2019-06-21 15:33:15.680000,False,False,False,False,False,False,False,False,False,False,False
2,vw_pc_type_II_diabetes,286624064,None,1,0,V,VIEW,2019-08-06 20:39:25.650000,2019-08-06 20:39:25.650000,False,False,False,False,False,False,False,False,False,False,False
3,vw_pc_type_I_diabetes,334624235,None,1,0,V,VIEW,2019-08-08 20:48:33.433333,2019-08-08 20:48:33.433333,False,False,False,False,False,False,False,False,False,False,False
4,vw_pc_dm_demo,338100245,None,1,0,V,VIEW,2019-06-07 19:42:59.906666,2019-06-21 15:22:15.306666,False,False,False,False,False,False,False,False,False,False,False
5,vw_pc_cancer_demo,354100302,None,1,0,V,VIEW,2019-06-21 02:44:01.136666,2019-06-21 15:24:05.716666,False,False,False,False,False,False,False,False,False,False,False
6,vw_pc_asthma_demo,370100359,None,1,0,V,VIEW,2019-06-21 03:26:03.723333,2019-06-21 14:50:30.363333,False,False,False,False,False,False,False,False,False,False,False
7,vw_pc_eds_demo,386100416,None,1,0,V,VIEW,2019-06-21 15:15:17.083333,2019-06-21 15:15:17.083333,False,False,False,False,False,False,False,False,False,False,False
8,vw_pc_copd_demo,402100473,None,1,0,V,VIEW,2019-06-21 15:45:32.433333,2019-06-21 15:45:32.433333,False,False,False,False,False,False,False,False,False,False,False
9,vw_pc_ami_demo,706101556,None,7,0,V,VIEW,2019-06-26 18:16:24.680000,2019-06-26 18:16:24.680000,False,False,False,False,False,False,False,False,False,False,False


In [7]:
query = "select count(*) from [dbo].[vw_pc_copd_demo]"
pd.read_sql_query(query,engine)

,
0,11919


In [8]:
from getSubdemographicsTables import getSubdemographicsTables

In [9]:
(df_labs_full, df_meds_full, df_procedures_full, 
 df_diagnoses_full, df_phenotypes_full) = getSubdemographicsTables(user, passwd, cohort='vw_pc_copd_demo')

In [10]:
from calculateAnyProfile import calculateAnyProfile

In [ ]:
(labs_counts, labs_frequencyPerYear, labs_fractionOfSubjects, labs_units, labs_names,
labs_stats, labs_aboveBelowNorm, labs_correlatedLabsCoefficients, labs_abscorrelation,
labs_correlatedMedsCoefficients, labs_correlatedProceduresCoefficients, 
 labs_correlatedDiagnosisCoefficients, 
 labs_correlatedPhenotypesCoefficients) = calculateAnyProfile('labs', df_labs=df_labs_full, 
                                                              df_meds=df_meds_full, df_procedures=df_procedures_full, 
                                                              df_diagnoses=df_diagnoses_full, 
                                                              df_phenotypes=df_phenotypes_full)

In [ ]:
from writeLabProfile import writeLabProfile

In [ ]:
writeLabProfile(labs_counts=labs_counts, labs_frequencyPerYear=labs_frequencyPerYear, labs_fractionOfSubjects=labs_fractionOfSubjects,
                    labs_units=labs_units, labs_names=labs_names,
                   labs_stats=labs_stats, labs_aboveBelowNorm=labs_aboveBelowNorm, 
                    labs_correlatedLabsCoefficients=labs_correlatedLabsCoefficients, labs_abscorrelation=labs_abscorrelation,
                 labs_correlatedMedsCoefficients=labs_correlatedMedsCoefficients,
                    labs_correlatedProceduresCoefficients=labs_correlatedProceduresCoefficients, 
                 labs_correlatedDiagnosisCoefficients=labs_correlatedDiagnosisCoefficients, 
                labs_correlatedPhenotypesCoefficients=labs_correlatedPhenotypesCoefficients,
                    cohort='copd', sex='All', race='All', age_low='All', 
                 age_high=None, topN=10, correlationCutoff=0.3)

In [12]:
(meds_medication, meds_dosageInfo, meds_frequencyPerYear, meds_fractionOfSubjects,
meds_correlatedLabsCoefficients, meds_correlatedDiagsCoefficients, 
 meds_correlatedMedsCoefficients,
meds_correlatedProceduresCoefficients, 
 meds_correlatedPhenotypesCoefficients) = calculateAnyProfile('medications', df_labs=df_labs_full, 
                                                              df_meds=df_meds_full, df_procedures=df_procedures_full, 
                                                              df_diagnoses=df_diagnoses_full, 
                                                              df_phenotypes=df_phenotypes_full)

In [ ]:
from writeMedProfile import writeMedProfile

In [ ]:
writeMedProfile(meds_medication, meds_frequencyPerYear, meds_fractionOfSubjects, 
                    meds_correlatedLabsCoefficients, meds_correlatedMedsCoefficients, 
                    meds_correlatedProceduresCoefficients, meds_correlatedDiagsCoefficients,
                    meds_correlatedPhenotypesCoefficients,
                    cohort='copd', sex='All', race='All', age_low='All', age_high=None,
                    topN=10, correlationCutoff=0.3)

In [ ]:
(diagnoses_code, diagnoses_count, diagnoses_frequencyPerYear, diagnoses_fractionOfSubjects,
diags_correlatedLabsCoefficients, diags_correlatedDiagsCoefficients, diags_correlatedMedsCoefficients,
diags_correlatedProceduresCoefficients, 
 diags_correlatedPhenotypesCoefficients) = calculateAnyProfile('diagnoses', df_labs=df_labs_full, 
                                                              df_meds=df_meds_full, df_procedures=df_procedures_full, 
                                                              df_diagnoses=df_diagnoses_full, 
                                                              df_phenotypes=df_phenotypes_full)

In [ ]:
from writeDiagProfile import writeDiagProfile

In [ ]:
writeDiagProfile(diagnoses_code, diagnoses_count, diagnoses_frequencyPerYear, diagnoses_fractionOfSubjects,
                    diags_correlatedLabsCoefficients, diags_correlatedDiagsCoefficients, diags_correlatedMedsCoefficients,
                    diags_correlatedProceduresCoefficients, diags_correlatedPhenotypesCoefficients,
                     cohort='copd')

In [ ]:
(procedures_code,
    procedures_count,
    procedures_frequencyPerYear,
    procedures_fractionOfSubjects,
    procs_correlatedLabsCoefficients,
    procs_correlatedDiagsCoefficients,
    procs_correlatedMedsCoefficients,
    procs_correlatedProceduresCoefficients,
    procs_correlatedPhenotypesCoefficients) = calculateAnyProfile('procedures', df_labs=df_labs_full, 
                                                              df_meds=df_meds_full, df_procedures=df_procedures_full, 
                                                              df_diagnoses=df_diagnoses_full, 
                                                              df_phenotypes=df_phenotypes_full)

In [ ]:
from writeProcProfile import writeProcProfile

In [ ]:
writeProcProfile(procedures_code,
    procedures_count,
    procedures_frequencyPerYear,
    procedures_fractionOfSubjects,
    procs_correlatedLabsCoefficients,
    procs_correlatedDiagsCoefficients,
    procs_correlatedMedsCoefficients,
    procs_correlatedProceduresCoefficients,
    procs_correlatedPhenotypesCoefficients,
    cohort='copd')

In [ ]:
(phenotypes_code, phenotypes_count, phenotypes_frequencyPerYear, phenotypes_fractionOfSubjects,
phenos_correlatedLabsCoefficients, phenos_correlatedDiagsCoefficients, 
phenos_correlatedMedsCoefficients,
phenos_correlatedProceduresCoefficients, 
 phenos_correlatedPhenotypesCoefficients) = calculateAnyProfile('phenotypes', df_labs=df_labs_full, 
                                                              df_meds=df_meds_full, df_procedures=df_procedures_full, 
                                                              df_diagnoses=df_diagnoses_full, 
                                                              df_phenotypes=df_phenotypes_full)

In [ ]:
from writeHPOProfile import writeHPOProfile

In [ ]:
writeHPOProfile(phenotypes_code, phenotypes_count, phenotypes_frequencyPerYear, phenotypes_fractionOfSubjects,
                    phenos_correlatedLabsCoefficients, phenos_correlatedDiagsCoefficients, phenos_correlatedMedsCoefficients,
                   phenos_correlatedProceduresCoefficients, phenos_correlatedPhenotypesCoefficients,
                     cohort='copd', sex='All', race='All', age_low='All', age_high=None,
                    topN=10, correlationCutoff=0.1)